In [2]:
!pip install -q transformers datasets

In [3]:
import pandas as pd

train_df = pd.read_json('../input/interview-type/comp_qu-cl_03_train.jsonl',  lines=True)
test_df = pd.read_json('../input/interview-type/comp_qu-cl_03_test.jsonl',  lines=True)
train_df.head(), test_df.head()

(                                              prompt  \
 0  What is the difference between local variables...   
 1  Considering the default SQL sandbox schema tha...   
 2  From a low-level perspective, design an URL sh...   
 3  What do you understand by the concept of micro...   
 4  How do you use the Django framework?\n$&$\nCod...   
 
                              completion  
 0                     ML Frameworks END  
 1                  Coding/Scripting END  
 2              Software Engineering END  
 3             Design & Architecture END  
 4  Language Internals and Framework END  ,
                                               prompt  \
 0  Explain about final, finally, and finalize() i...   
 1  Have you used NodeJS in any of your projects? ...   
 2  function add(a, b) { console.log('calling add ...   
 3  Given two strings s and t, determine if they a...   
 4  Do you know about the levels of cache of Hiber...   
 
                              completion  
 0  Langua

In [4]:
train_df['completion'] = train_df['completion'].str.replace('END', '')
test_df['completion'] = test_df['completion'].str.replace('END', '')
train_df.sample(5)

,prompt,completion
150478,How memory get segmented?\n$&$\nCoding & Probl...,Language Internals and Framework
66612,Create a react app to display random users in ...,JS frameworks
33337,Explain about framework you have used?\n$&$\nC...,Automation
80160,Briefly discuss one of your most interesting p...,Software Engineering
113747,You have been given an array of integers. You ...,Coding & Problem Solving


In [5]:
train_df['prompt'] = train_df['prompt'].str.replace('###', '')
test_df['prompt'] = test_df['prompt'].str.replace('###', '')

In [6]:
test_df.sample(5)

,prompt,completion
9847,"There is a given string. ""I love GitHub"" // ->...",Coding & Problem Solving
1643,Briefly explain the project you are currently ...,Design & Architecture
9890,Explain Spring Dependency Injection?\n$&$\nCod...,Language Internals and Framework
1815,Can we have an abstract class without an abstr...,Coding & Problem Solving
3065,How will the matrix come to the system and CPU...,Monitoring & Alerting


In [7]:
len(train_df.completion.unique())

154

In [8]:
train_df.describe()

,prompt,completion
count,206977,206977
unique,206582,154
top,Build a simple pager duty application Product ...,Coding & Problem Solving
freq,8,24497


In [9]:
train_df.to_csv('comp_qu-cl_03_train.csv', index=False)
test_df.to_csv('comp_qu-cl_03_test.csv', index=False)

In [10]:
from datasets import load_dataset

data_files = {
    'train': './comp_qu-cl_03_train.csv',
    'test' : './comp_qu-cl_03_test.csv'
}
ds = load_dataset('csv', data_files = data_files)
ds

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-79d3ef053d870190/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 206977
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 10896
    })
})

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name='distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 154,ignore_mismatched_sizes=True)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([154, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([154]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
end_row = test_df[test_df['completion'].str.contains('END')]
end_row

,prompt,completion


In [13]:
from datasets import ClassLabel
classes=train_df.completion.unique()
labels=ClassLabel(num_classes=154, names=classes)
classes

array(['ML Frameworks ', 'Coding/Scripting ', 'Software Engineering ',
       'Design & Architecture ', 'Language Internals and Framework ',
       'Application Development ',
       'Application Design (Low Level Design) ', 'SDLC / STLC ',
       'Statistics ', 'Coding & Problem Solving ', 'Tools ',
       'Problem Solving and Technical Expertise ', 'JS frameworks ',
       'Scripting ', 'React Native ', 'Automation ',
       'Containerisation & Orchestration ', 'JSFrameworks ',
       'Scaling Techniques ', 'Backend/API Testing ', 'Javascript ',
       'System & Networking ', 'HTML-CSS ', 'Data Pipeline & Scripting ',
       'Low Level Design ', 'Cloud Computing ', 'Browser internals ',
       'Big Data Technologies ', 'Coding & Problem Solving-Backend ',
       'Debugging ', 'App Development ',
       'Test Case/ Test Plan  Enumeration ',
       'Design & Architecture - Backend ', 'Low Level Design(BE) ',
       'Decoupling of Systems (High Level Design) ', 'SQL ',
       'Architect

In [14]:
ds_clean = ds['train'].train_test_split(test_size = 0.1, seed=42)
ds_clean['validation'] = ds_clean.pop('test')
ds_clean['test'] = ds['test']
ds_clean

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 186279
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 20698
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 10896
    })
})

In [15]:
from transformers import DataCollatorWithPadding

def tokenize(example):
    example['labels']=labels.str2int(example['completion'])
    return tokenizer(example['prompt'], truncation=True)

tokenized_ds = ds_clean.map(tokenize, batched=True)
collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/187 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [16]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 186279
    })
    validation: Dataset({
        features: ['prompt', 'completion', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 20698
    })
    test: Dataset({
        features: ['prompt', 'completion', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 10896
    })
})

In [17]:
tokenized_ds = tokenized_ds.remove_columns(['prompt', 'completion'])
tokenized_ds.set_format('torch')
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 186279
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 20698
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 10896
    })
})

In [18]:
tokenized_ds['train'][0]

{'labels': tensor(93),
 'input_ids': tensor([  101,  2129,  2017,  2024,  2183,  2000,  3413,  1996,  5310, 18442,
          1029,  1002,  1004,  1002, 16861,  1064,  3231,  3572,  1064, 29296,
          1064,  7705,  1064, 25022,  1013,  3729,  1064, 17371, 15472,  1013,
          2358, 15472,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [19]:
from torch.utils.data import DataLoader

train_loader=DataLoader(tokenized_ds['train'], shuffle=True, batch_size=16, collate_fn=collator)
test_loader=DataLoader(tokenized_ds['test'], collate_fn=collator)
val_loader=DataLoader(tokenized_ds['validation'], collate_fn=collator)

In [20]:
import torch
from transformers import get_scheduler

device = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_EPOCHS=5
NUM_TRAINING_STEP=NUM_EPOCHS* len(train_loader)

optim=torch.optim.AdamW(model.parameters(), lr=5e-5)
lr_scheduler=get_scheduler(name='linear', optimizer=optim, num_training_steps=NUM_TRAINING_STEP, num_warmup_steps=0)

In [21]:
for batch in train_loader:
    break
{k:v.shape for k, v in batch.items()}

{'labels': torch.Size([16]),
 'input_ids': torch.Size([16, 259]),
 'attention_mask': torch.Size([16, 259])}

In [22]:
from transformers import get_scheduler
import torch

optim = torch.optim.AdamW(model.parameters(), lr = 5e-5)
device = 'cuda' if torch.cuda.is_available() else 'cpu'


num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optim,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

34929


In [23]:
criteria = torch.nn.CrossEntropyLoss()

In [24]:
import numpy as np

In [25]:
train_df['completion'].value_counts()

Coding & Problem Solving             24497
Design & Architecture                16859
Software Engineering                 12930
Language Internals and Framework     11659
Javascript                           10239
                                     ...  
Data Extraction                        101
Past Experience                         99
Android                                 99
PHP Internals                           98
Software Engineering (BE)               97
Name: completion, Length: 154, dtype: int64

In [26]:
!pip install -q wandb
import wandb
!wandb login

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [28]:
import wandb

wandb.init(project="DistilBert-SST-2", entity="justalearner")

wandb: Currently logged in as: smartpy (justalearner). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
wandb.watch(model)

[]

In [30]:
print(device)

cuda


In [33]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

Num_epochs = 1

model.to(device)
progress_bar = tqdm(train_loader, desc='Training')
train_loss_per_epoch, val_loss_per_epoch, train_acc_per_epoch, val_acc_per_epoch = [], [], [], []
for epoch in tqdm(range(Num_epochs)):
    model.train()
    batch_train_losses, batch_train_acc=[], []
    for batch in progress_bar:
        batch = {k:v.to(device) for k, v in batch.items()}
        output = model(input_ids = batch['input_ids'], attention_mask=batch['attention_mask']
                      )
        loss = criteria(output.logits, batch['labels'])    
        acc = accuracy_score(batch['labels'].cpu().numpy(), output.logits.argmax(dim=1).cpu().numpy())
        loss.backward()
        optim.step()
        lr_scheduler.step()
        optim.zero_grad()
        batch_train_losses.append(loss.item())
        batch_train_acc.append(acc)
    train_loss_per_epoch.append(np.mean(batch_train_losses))
    avg_loss_per_batch = np.mean(batch_train_losses)
    progress_bar.set_description(f'loss on epoch {epoch}:{avg_loss_per_batch}, acc:{np.mean(batch_train_acc)}')
    train_loss_per_epoch.append(avg_loss_per_batch)

    model.eval()
    with torch.no_grad():
        batch_val_losses, batch_val_acc = [], []
        for batch in val_loader:
            batch = {k:v.to(device) for k, v in batch.items()}
            output = model(input_ids = batch['input_ids'], attention_mask=batch['attention_mask']
                          )
            loss = criteria(output.logits, batch['labels'])
            acc = accuracy_score(batch['labels'].cpu().numpy(), output.logits.argmax(dim=1).cpu().numpy())
            batch_val_losses.append(loss.item())
            batch_val_acc.append(acc)
        val_loss_per_epoch.append(np.mean(batch_val_losses))
        val_acc_per_epoch.append(np.mean(batch_val_acc))
        progress_bar.set_description(f'val loss on epoch {epoch}:{np.mean(batch_val_losses)}, acc:{np.mean(batch_val_acc)}')
        wandb.log({'train_loss': avg_loss_per_batch, 'val_loss': np.mean(batch_val_losses), 'val_acc': np.mean(batch_val_acc)
        , 'train_acc': np.mean(batch_train_acc)})
        progress_bar.set_description(f'val loss on epoch {epoch}:{np.mean(batch_val_losses)}, acc:{np.mean(batch_val_acc)}')

        

Training:   0%|          | 0/11643 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
model.save_pretrained('./model')
tokenizer.save_pretrained('./tokenizer')

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

In [35]:
from datasets import load_metric

precision = load_metric('precision')
recall = load_metric('recall')
f1 = load_metric('f1')

model.eval()
for batch in tqdm(val_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    f1.add_batch(predictions=predictions, references=batch["labels"])
    precision.add_batch(predictions=predictions, references=batch["labels"])
    recall.add_batch(predictions=predictions, references=batch["labels"])

print(f'f1 : ', f1.compute(average='macro'))
print(f'precision : ', precision.compute(average='macro'))
print(f'recall : ', recall.compute(average='macro'))

  0%|          | 0/20698 [00:00<?, ?it/s]

f1 :  {'f1': 0.7341052535158318}
precision :  {'precision': 0.7524008049063318}
recall :  {'recall': 0.737186755504466}


In [36]:
from datasets import load_metric

precision = load_metric('precision')
recall = load_metric('recall')
f1 = load_metric('f1')
preds = []
model.eval()
for batch in tqdm(test_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    preds.append(predictions.cpu().numpy())
    f1.add_batch(predictions=predictions, references=batch["labels"])
    precision.add_batch(predictions=predictions, references=batch["labels"])
    recall.add_batch(predictions=predictions, references=batch["labels"])

print(f'f1 : ', f1.compute(average='macro'))
print(f'precision : ', precision.compute(average='macro'))
print(f'recall : ', recall.compute(average='macro'))

  0%|          | 0/10896 [00:00<?, ?it/s]

f1 :  {'f1': 0.7282605971324185}


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision :  {'precision': 0.7415409976617591}
recall :  {'recall': 0.731974742857981}


In [ ]:
torch.save(model.state_dict(), 'model.pt')

In [37]:
test_df['prompt'] = test_df['prompt'].apply(lambda x:x.replace('\n', ' '))
train_df['prompt'] = train_df['prompt'].apply(lambda x:x.replace('\n', ' '))
test_df['preds'] = test_df['prompt'].apply(lambda x: x.split('$&$')[1].split('|'))
train_df['preds'] = train_df['prompt'].apply(lambda x: x.split('$&$')[1].split('|'))
train_df['completion'] = train_df['completion'].str.replace('END', '')
test_df['completion'] = test_df['completion'].str.replace('END', '')
test_df.head()

,prompt,completion,preds
0,"Explain about final, finally, and finalize() i...",Language Internals and Framework,"[ Coding & Problem Solving, Language Internals..."
1,Have you used NodeJS in any of your projects? ...,NodeJS,"[ Coding & Problem Solving, NodeJS, Design & A..."
2,"function add(a, b) { console.log('calling add ...",Design & Architecture,"[ Design & Architecture, Coding & Problem Solv..."
3,"Given two strings s and t, determine if they a...",Coding,"[ Coding, SQL, Automation, CI/CD, Technical Br..."
4,Do you know about the levels of cache of Hiber...,Language Internals and Framework,"[ Coding & Problem Solving, Language Internals..."


In [38]:
def list_formatting(li):
    new_li = []
    for word in li:
        word = word.strip()
        word = word + ' '
        new_li.append(word)
    return new_li


test_df['new_preds'] = test_df['preds'].apply(list_formatting) 
test_df.head()

,prompt,completion,preds,new_preds
0,"Explain about final, finally, and finalize() i...",Language Internals and Framework,"[ Coding & Problem Solving, Language Internals...","[Coding & Problem Solving , Language Internals..."
1,Have you used NodeJS in any of your projects? ...,NodeJS,"[ Coding & Problem Solving, NodeJS, Design & A...","[Coding & Problem Solving , NodeJS , Design & ..."
2,"function add(a, b) { console.log('calling add ...",Design & Architecture,"[ Design & Architecture, Coding & Problem Solv...","[Design & Architecture , Coding & Problem Solv..."
3,"Given two strings s and t, determine if they a...",Coding,"[ Coding, SQL, Automation, CI/CD, Technical Br...","[Coding , SQL , Automation , CI/CD , Technical..."
4,Do you know about the levels of cache of Hiber...,Language Internals and Framework,"[ Coding & Problem Solving, Language Internals...","[Coding & Problem Solving , Language Internals..."


In [39]:
from tqdm.auto import tqdm
def predict_proba(sentence):
    model.to(device)
    model.eval()
    count = 0
    all_top_k = []
    progress_bar = tqdm(test_loader)
    idx = 0
    for batch in progress_bar:
        
        batch = {k: v.to(device) for k, v in batch.items()}
        desired_classes = test_df[idx:idx+16]['new_preds']

            
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)

        top_k = {}
        for i in range(len(test_df.iloc[idx]['new_preds'])):
            try:
                class_label = labels.str2int(test_df.iloc[idx]['new_preds'][i])
                top_k[test_df.iloc[idx]['new_preds'][i]] = probs[0, class_label]
            except:
                top_k[i] = 0
        all_top_k.append(top_k)
        progress_bar.update(1)
        idx += 1
    return all_top_k
all_top_k = predict_proba(test_df['prompt'].tolist())

  0%|          | 0/10896 [00:00<?, ?it/s]

In [40]:
all_top_k_cpu = []

for li in all_top_k:
    temp = {}
    for k, v in li.items():
        if not isinstance(v, int) and not isinstance(k, int):
            temp[k] = v.cpu().numpy()
    
    all_top_k_cpu.append(temp)
all_top_k_cpu[0]

{'Coding & Problem Solving ': array(0.00412005, dtype=float32),
 'Language Internals and Framework ': array(0.99297076, dtype=float32),
 'Application Design ': array(0.00148673, dtype=float32),
 'Service Oriented Architecture ': array(0.00061292, dtype=float32)}

In [41]:
from collections import OrderedDict

top_ks = []
for li in all_top_k_cpu:
    max_prob = 0
    for k, v in li.items():
        if max_prob < v:
            max_prob = v
            max_word = k
    top_ks.append(max_word)
top_ks[0]

'Language Internals and Framework '

In [42]:
df_final=pd.DataFrame({
    'Prompt':test_df['prompt'],
    'Completion':test_df['completion'],
    'Prediction': top_ks,
    'Top-K':all_top_k_cpu
})
df_final.shape

(10896, 4)

In [43]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
import pandas as pd
import numpy as np

def metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    precision = precision_score(actual, predicted, average='macro')
    recall = recall_score(actual, predicted, average='macro')
    f1 = f1_score(actual, predicted, average='macro')
    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1:', f1)
    return accuracy, precision, recall, f1

def confusion_metrics(actual, predicted):
    cm = confusion_matrix(actual, predicted)
    cr = classification_report(actual, predicted)
    print('Confusion Matrix:')
    print(cm)
#     plt.figure(figsize=(10, 7))
#     sn.heatmap(cm, annot=True, fmt='d', cmap='Blues')
#     plt.ylabel('Actual')
#     plt.xlabel('Predicted')
#     plt.show()
    print('Classification Report:')
    print(cr)
    return cm, cr
confusion_metrics(df_final['Completion'], df_final['Prediction'])
metrics(df_final['Completion'], df_final['Prediction'])

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:
[[  7   0   0 ...   0   0   0]
 [  0  17   0 ...   0   0   0]
 [  0   0   4 ...   0   0   0]
 ...
 [  0   0   0 ...   7   0   0]
 [  0   0   0 ...   0   9   0]
 [  0   0   0 ...   0   0 110]]
Classification Report:
                                                    precision    recall  f1-score   support

                                   API  Designing        0.78      0.70      0.74        10
                     API Automation and Framework        1.00      1.00      1.00        17
                                          Android        1.00      1.00      1.00         4
             App Architecture & High Level Design        0.83      0.89      0.86        27
                                  App Development        0.67      0.52      0.59       107
           App Development, Architecture & Design        0.64      0.76      0.70        59
                                      App Release        0.70      0.78      0.74         9
                              

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8008443465491923
Precision: 0.7444097953070715
Recall: 0.7346769719823153
F1: 0.7310028786051108


(0.8008443465491923,
 0.7444097953070715,
 0.7346769719823153,
 0.7310028786051108)

In [45]:
acc, pre, rec, f1 = metrics(df_final['Completion'], df_final['Prediction'])
cr = classification_report(df_final['Completion'], df_final['Prediction'])
with open('report_without_trunc.txt', 'w') as f:
    f.write(str(cr))
with open('metric_without_trunc.txt', 'w') as f:
    f.write('Accuracy'+str(acc))
    f.write('Precision'+str(pre))
    f.write('Recall'+str(rec))
    f.write('F1'+str(f1))

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.8008443465491923
Precision: 0.7444097953070715
Recall: 0.7346769719823153
F1: 0.7310028786051108


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def predict(sentence):
    tokens=tokenizer(sentence, truncation=True, max_length=128, padding='max_length', return_tensors='pt')
    tokens={k:v.to(device) for k, v in tokens.items()}
    output=model(**tokens)
    logits=output.logits
    prediction=torch.argmax(logits, axis=-1)
    return prediction.item()

ran_num = np.random.randint(0, len(test_df))
print(f'Actual :', test_df['completion'][ran_num])
print('Prediction :', labels.int2str(predict(test_df['prompt'][ran_num])))

In [ ]:
preds_labels = []
for i in range(len(test_df)):
    preds_labels.append(labels.int2str(preds[i]))

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(test_df['completion'], preds_labels)
print(classification_report(test_df['completion'], preds_labels))

In [ ]:
with open('report.txt', 'w') as f:
    f.write(str(report))

### cm


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

plt.figure(figsize=(100, 100))
preds_labels = []
for i in range(len(test_df)):
    preds_labels.append(labels.int2str(preds[i]))
c_f=confusion_matrix(test_df['completion'], preds_labels)
# disp = ConfusionMatrixDisplay(confusion_matrix=c_f,
#                              display_labels=classes)
# disp.plot()
# plt.show()